## Import package

In [3]:
import sys
sys.path.append('/home/sun/ETGEMS-10.20/ET_OptME/script')
# import ETGEMs_function_protain as etgf
from ETGEMs_function_protain import *
from ET_optme import *
import pandas as pd
import cobra
import gurobipy
import json
import multiprocessing
import os
from multiprocessing import Pool
from tqdm import tqdm 
from sympy import subsets
import pandas as pd
import matplotlib.pyplot as plt
import re

## setting path for models and data, example: using C.glutamicum to produce L_Isoleucine

In [ ]:
# data 
model_file="./data/lys.json" # c.glutamicum model, irreversible and reactions with isoezyme splited
reaction_kcat_MW_file='./data/reaction_change_by_enzuse_PDH_n.csv'
# results
path_results = './result'
savepath = './result/picture'
dictionarymodel_path='./result/dictionarymodel_iCW.json' # dictionary model saving path
wild_reaction_flux_path='./result/reaction centered/wild-enzyme-reaction_e0_ile10.27.json'
over_reaction_flux_path='./result/reaction centered/over-enzyme-reaction_e0_ile10.27.json'
wild_enzyme_concentration_path = './result/ET_Optme/result/ile/wild-enzyme.json'
over_enzyme_concentration_path = './result/ile/over-enzyme.json'
# case study
model0_file = './task/iCW773_uniprot_modification_del.json'
reaction_g0_file=os.path.join('./task/iCW773_uniprot_modification_del_reaction_g0.csv')
metabolites_lnC_file = os.path.join('./task/metabolites_lnC_cg1.txt')

## setting task, prepare task.json

In [6]:
# model、substrate、product、oxygence
# mode: SET
inputdic = {"model":'lys.json',
"substrate":"EX_glc_e_reverse",
"biomass": "CG_biomass_cgl_ATCC13032",
 "product": "EX_ile_L_e",
"taskname": "optforce",
"mode":"SET",
"oxygenstate":"aerobic"}
# save task.json
with open('/home/sun/ETGEMS-10.20/ET_OptME/data/task.json', 'w') as json_file:
    json.dump(inputdic, json_file, indent=4)
path_strain = 'iCW'

## invoke ET-OptME and run the task 

## 1.enzyme-thermo Model Construction

In [9]:
# read model
model = cobra.io.load_json_model(model_file)

# Store model data into Get_Concretemodel_Need_Data
Concretemodel_Need_Data = Get_Concretemodel_Need_Data(model_file)

#model0 = cobra.io.load_json_model(model0_file)

# Save the protein-centered model as a dictionary
dictionary_model = json_load(dictionarymodel_path)
dictionary_model.keys()

rname3 = []

# Store the protein-centered dictionary into Get_Concretemodel_Need_Data
get_dictionarymodel_data2(dictionary_model, Concretemodel_Need_Data, rname3)

# Store thermodynamic data into Get_Concretemodel_Need_Data
Get_Concretemodel_Need_Data_g0(Concretemodel_Need_Data, reaction_g0_file, metabolites_lnC_file, reaction_kcat_MW_file)

Concretemodel_Need_Data['reaction_g0']['g0'] = Concretemodel_Need_Data['reaction_g0']['g0'].replace(0, np.nan)
Concretemodel_Need_Data['reaction_g0'].dropna(subset=['g0'], inplace=True)

Concretemodel_Need_Data['reaction_g0'].at['AIRC3_reverse', 'g0'] = 0
Concretemodel_Need_Data['reaction_g0'].at['ATPS4rpp_reverse_num2', 'g0'] = 0

Inc = Concretemodel_Need_Data['metabolites_lnC']

for i in Concretemodel_Need_Data['metabolite_list']:
    if i not in Inc.index:
        Inc.loc[i, 'lnClb'] = -14.508658
        Inc.loc[i, 'lnCub'] = -3.912023

Concretemodel_Need_Data['metabolites_lnC'] = Inc

# Modify the oxygen conditions of the model
if inputdic['oxygenstate'] == 'aerobic':
    model.reactions.get_by_id('EX_o2_e_reverse').upper_bound = 1000
if inputdic['oxygenstate'] == 'micro_aerobic': 
    model.reactions.get_by_id('EX_o2_e_reverse').upper_bound = 2 
if inputdic['oxygenstate'] == 'anaerobic': 
    model.reactions.get_by_id('EX_o2_e_reverse').upper_bound = 0



## 2.Algorithm :  ET-OptForceM

In [5]:
Concretemodel_Need_Data

{'model': <Model None at 0x7f934b5ea640>,
 'reaction_list': ['ALAR',
  'ALATA_L',
  'ASNN',
  'ASNS1',
  'ASNS2',
  'ASPT',
  'ASPTA',
  'DAAD',
  'VPAMT',
  '2METS',
  'ACHMSC',
  'ACNML',
  'ADNUC_num1',
  'AGDC',
  'ALCD19_num1',
  'ALDD19x',
  'ALDD2x',
  'AMALT1',
  'AMALT2',
  'AMALT3',
  'AMALT4',
  'AMANAPEr',
  'APG',
  'ARAI_num1',
  'BETALDHx',
  'BETALDHy',
  'CBMKr',
  'DHACOAH',
  'DHAPT',
  'DHPTDNRN',
  'DRPA',
  'FFSD_num1',
  'FRUK',
  'G3PD2',
  'G3PT',
  'G6PDA',
  'GLCATr',
  'GLCRD',
  'GLYCDx',
  'GLYK',
  'GNK_num1',
  'GUI1',
  'HCO3E',
  'HEX7',
  'HPYRI_num1',
  'HSST_2',
  'HXCT',
  'MALGT',
  'MALTATr',
  'MAN1PG',
  'MAN6PI',
  'MCITL2_num1',
  'MM_COA_ADD5',
  'MMM2',
  'MSDH_num1',
  'PFK_2',
  'PGLYCP_num1',
  'PGMT',
  'PMANM_num1',
  'PPASUCC',
  'RBK_num1',
  'SUCptsp',
  'TRE6p',
  'TRE6PP',
  'TRE6PS',
  'UACGAMPpp',
  'UDPARA',
  'UDPG4E',
  'UDPGALPpp',
  'UDPGPpp_num1',
  'UGLCURPpp',
  'XYLI1',
  'XYLI2',
  'XYLK',
  'GALKr',
  'M1PD',
  'MAN2D

In [10]:
# Calculate the maximum growth rate #打印出数值的解释和单位
B_value1,v0_biomass,bio,totalE,objvalue2 = calculate_biomass(Concretemodel_Need_Data,inputdic,model,'iCW')

maximum growth rate value is: 0.4287238602214586(mmol/gDW/h)
The maximum thermodynamic driving force value is : 0.4287238602214586
minimizing the total enzyme concentration  is: 0.12121613628831285((mmol/gDW)


## 2.1 Reaction Centered

In [11]:
# The enzymes that catalyze each reaction in the enzyme-thermo model
reaction_list=Concretemodel_Need_Data['e0']
obj_names = reaction_list
pool = multiprocessing.Pool(processes=multiprocessing.cpu_count())
# Based on the range of enzyme changes for each reaction in reference type
results = pool.starmap(calculate_wildrange_reaction, [(Concretemodel_Need_Data,obj_name,inputdic,objvalue2,v0_biomass,totalE,path_strain,B_value1) for obj_name in obj_names])

pool.close()
pool.join()


final_results = {}
for result in results:
    final_results.update(result)

# Save the results as a JSON file
wild_file_path = os.path.join(wild_reaction_flux_path)
with open(wild_file_path, 'w') as json_file:
    json.dump(final_results, json_file)

Read LP format model from file /tmp/tmpqn7pry71.lp
Reading time = 0.07 seconds
: 949 rows, 3704 columns, 15400 nonzeros
Read LP format model from file /tmp/tmpitiug7q2.lp
Reading time = 0.03 seconds
Read LP format model from file /tmp/tmp472nyd5e.lp
Reading time = 0.04 seconds
: 949 rows, 3704 columns, 15400 nonzeros
: 949 rows, 3704 columns, 15400 nonzeros
Read LP format model from file /tmp/tmp262drov3.lp
Reading time = 0.04 seconds
: 949 rows, 3704 columns, 15400 nonzeros
Read LP format model from file /tmp/tmpcow4y70z.lp
Reading time = 0.03 seconds
: 949 rows, 3704 columns, 15400 nonzeros
Read LP format model from file /tmp/tmptyviju26.lp
Reading time = 0.03 seconds
: 949 rows, 3704 columns, 15400 nonzeros
Read LP format model from file /tmp/tmp7rdx0x32.lp
Reading time = 0.03 seconds
: 949 rows, 3704 columns, 15400 nonzeros
Read LP format model from file /tmp/tmpgz9qxvzg.lp
Reading time = 0.03 seconds
: 949 rows, 3704 columns, 15400 nonzeros
Read LP format model from file /tmp/tmp9

TypeError: FBA_template2() got an unexpected keyword argument 'set_obj_value_e0'

In [ ]:
# Calculate the maximum production rate
B_value2,v1_product_max,pro,totalE2,EcoECM_FBA_protainmodel_B2 = calculate_product(Concretemodel_Need_Data,inputdic,objvalue2,path_strain,v0_biomass)

maximum growth rate value is: 3.150759311958292(mmol/gDW/h)
The maximum thermodynamic driving force value is : 3.150759311958292
minimizing the total enzyme concentration  is: 0.07784801790914309((mmol/gDW)


In [ ]:
# The enzymes that catalyze each reaction in the model
reaction_list=Concretemodel_Need_Data['e0']

obj_names = reaction_list
pool = multiprocessing.Pool(processes=multiprocessing.cpu_count())

results = pool.starmap(calculate_over_reaction, [(Concretemodel_Need_Data,obj_name,inputdic,objvalue2,v0_biomass,v1_product_max,totalE2,path_strain,B_value2) for obj_name in obj_names])
pool.close()
pool.join()


final_results = {}
for result in results:
    final_results.update(result)
over_file_path = os.path.join(over_reaction_flux_path)
with open(over_file_path, 'w') as json_file:
    json.dump(final_results, json_file)

## 2.2 Protein Centered

In [ ]:
# Calculate the enzyme concentration range under maximum growth conditions
enzyme_list = list(Concretemodel_Need_Data['mw_dict'].keys()) 
obj_names = enzyme_list
pool = multiprocessing.Pool(processes=multiprocessing.cpu_count())

# Use multiprocessing to calculate the wild range for each enzyme
results = pool.starmap(calculate_wildrange, [(Concretemodel_Need_Data, obj_name, inputdic, objvalue2, v0_biomass, totalE, path_strain, B_value1) for obj_name in obj_names])

pool.close()
pool.join()

# Combine the results into a final dictionary
final_results = {}
for result in results:
    final_results.update(result)

# Save the results as a JSON file
wild_file_path = os.path.join(wild_enzyme_concentration_path)
with open(wild_file_path, 'w') as json_file:
    json.dump(final_results, json_file)



In [ ]:
# The enzymes that catalyze each reaction in the model
enzyme_list = list(Concretemodel_Need_Data['mw_dict'].keys()) 
obj_names = enzyme_list
pool = multiprocessing.Pool(processes=multiprocessing.cpu_count())

results = pool.starmap(calculate_over, [(Concretemodel_Need_Data,obj_name,inputdic,objvalue2,v0_biomass,v1_product_max,totalE2,path_strain,B_value2) for obj_name in obj_names])
pool.close()
pool.join()


final_results = {}
for result in results:
    final_results.update(result)
over_file_path = os.path.join(over_enzyme_concentration_path)
with open(over_file_path, 'w') as json_file:
    json.dump(final_results, json_file)

Read LP format model from file /tmp/tmpfqjlbr9r.lp
Reading time = 0.02 seconds
: 949 rows, 3704 columns, 15400 nonzeros
Read LP format model from file /tmp/tmpzj2siohw.lp
Reading time = 0.03 seconds
: 949 rows, 3704 columns, 15400 nonzeros
Read LP format model from file /tmp/tmpyjpbivh4.lp
Reading time = 0.02 seconds
: 949 rows, 3704 columns, 15400 nonzeros
Read LP format model from file /tmp/tmp7ngwg8ul.lp
Reading time = 0.02 seconds
: 949 rows, 3704 columns, 15400 nonzeros
Read LP format model from file /tmp/tmpzjg6xrdd.lp
Reading time = 0.02 seconds
: 949 rows, 3704 columns, 15400 nonzeros
Read LP format model from file /tmp/tmpyo5x8we7.lp
Reading time = 0.02 seconds
: 949 rows, 3704 columns, 15400 nonzeros
Read LP format model from file /tmp/tmpw46v3l0m.lp
Reading time = 0.02 seconds
: 949 rows, 3704 columns, 15400 nonzeros
Read LP format model from file /tmp/tmpbhvb8rdq.lp
Reading time = 0.02 seconds
: 949 rows, 3704 columns, 15400 nonzeros
Read LP format model from file /tmp/tmpo

## Analyze the result and generate table and figure(range compare, flux vs prouct) 

## 1.Reaction Centered

In [ ]:
#Read result file
enzyme_results_data,enzyme_overresults2_data = read_file_rea(path_results)

In [ ]:
# Filter the targets that need modification
ko_data, up_data, down_data = compare_results(enzyme_results_data, enzyme_overresults2_data, inputdic)

In [ ]:
# Generate the corresponding equations for the reactions
equation_dict = gene_reaction_map1(reaction_list, model)
# Generate the GPR (Gene-Protein-Reaction) relationships for the reactions
gpr_dict = gpr_map(reaction_list, model)
# Generate the results table (modification strategies, reaction equations, reaction GPR relationships, etc.)
wild_data = [{'reaction':reaction, 'flux_wild':[format(value, '.3e') for value in data['range']]} for reaction, data in enzyme_results_data.items()]
df1 = pd.DataFrame(wild_data)
over_data = [{'reaction':reaction, 'flux_over':[format(value, '.3e') for value in data['range']]} for reaction, data in enzyme_overresults2_data.items()]
df2 = pd.DataFrame(over_data)
merged_df = pd.merge(df1, df2, on='reaction', how='inner')
merged_df['equation'] = merged_df['reaction'].map(equation_dict)
merged_df['gene'] = merged_df['reaction'].map(gpr_dict)
# Fill in the 'manipulations' column with None initially
merged_df['manipulations'] = None

for reaction, _ in ko_data:
    merged_df.loc[merged_df['reaction'] == reaction, 'manipulations'] = 'ko'
for reaction, _ in up_data:
    merged_df.loc[merged_df['reaction'] == reaction, 'manipulations'] = 'Up'
for reaction, _ in down_data:
    merged_df.loc[merged_df['reaction'] == reaction, 'manipulations'] = 'down'

# Generate the output results file
KeyInfo = pd.DataFrame()
bio_df = pd.DataFrame()
pro_df = pd.DataFrame()
Df = pd.DataFrame()
output_rea_e0(KeyInfo, path_results, bio_df, pro_df, merged_df, inputdic, Df)


## 2.Protain Centered

In [ ]:
#Read results file
enzyme_results_data,enzyme_overresults2_data = read_file(path_results)

In [ ]:
# Filter the targets that need modification
ko_data,up_data,down_data = compare_results(enzyme_results_data, enzyme_overresults2_data,inputdic)

In [ ]:
# get reaction equation
equation_dict = gene_reaction_map1(reaction_list,model)

In [ ]:
# Enzyme concentrations and flux distribution under the wild-type condition
E_refdict, mw_dict, reaction_dict_bio, kcat_dict = ref_e_con(inputdic, Concretemodel_Need_Data, totalE, path_strain, objvalue2)


0.4072876672083201


In [ ]:
# Retrieve gene, equation, and reaction mappings
new_gene_reaction_mapping_bio, gene_reaction_mapping, gene_kcat_mapping = gene_reaction_map(enzyme_list, reaction_dict_bio, kcat_dict, Concretemodel_Need_Data, model)


## get reaction flux

In [ ]:
# Retrieve the reaction flux distribution under maximum production conditions
reaction_dict, kcat_dict, EcoECM_FBA_protainmodel_pro_max = reaction_flux(inputdic, Concretemodel_Need_Data, totalE2, path_strain, totalE, objvalue2, v0_biomass)

2.740861572132984


In [ ]:
# Retrieve the enzyme concentrations and corresponding reaction flux distributions for the overexpressed state
new_gene_reaction_mapping, gene_reaction_mapping, gene_kcat_mapping = gene_reaction_map(enzyme_list, reaction_dict, kcat_dict, Concretemodel_Need_Data, model)

In [ ]:
# Retrieve the enzyme usage proportions for each enzyme in the model
normalized_E_dict, E_dict, e_dict = enzyme_usage_over(EcoECM_FBA_protainmodel_pro_max, Concretemodel_Need_Data)


sum: 0.07605156126806689


In [ ]:
# Calculate the adjustment level for each modification strategy
Fold_change = fold_change(E_dict, E_refdict)

In [ ]:
# List of all reactions in the model
reaction_list = []
for rea in model.reactions:
    reaction_list.append(rea.id)

In [ ]:
# Retrieve the reaction equations
equation_dict = gene_reaction_map1(reaction_list, model)

In [ ]:
# Generate the result table (modification strategies, equations, adjustment levels, enzyme proportions, etc.)
meged_df = must_df(inputdic, equation_dict, new_gene_reaction_mapping_bio, new_gene_reaction_mapping, E_refdict, E_dict, Fold_change, normalized_E_dict, e_dict, gene_kcat_mapping, enzyme_results_data, enzyme_overresults2_data, ko_data, up_data, down_data, Concretemodel_Need_Data)


In [ ]:
# Overflow reactions in the model for wild and overexpressed states, along with biomass and product B value
bio_df, pro_df, KeyInfo = basic(bio, pro, B_value1, v0_biomass, B_value2, v1_product_max, totalE, totalE2, inputdic)


In [ ]:
# Identify bottleneck reactions under thermodynamic constraints
bottleneck_reactions_list, Df = bottleneck_reactions(EcoECM_FBA_protainmodel_B2, model)


In [ ]:
# Generate the result table
output(KeyInfo,path_results,bio_df,pro_df,meged_df,inputdic,Df)

## Compare reaction centered and protein centered 

## 1. reaction centered results

In [ ]:
# Read file
reaction_centered = pd.read_excel('/home/sun/ETGEMS-10.20/ET_OptME/result/reaction centered/results_reaction_e0.xlsx',sheet_name='MUST')
reaction_centered = reaction_centered.drop('Unnamed: 0',axis=1)
reaction_centered = reaction_centered[(reaction_centered['manipulations'] == 'down')|(reaction_centered['manipulations'] == 'Up')]

In [ ]:
# The strategies for multiple reactions catalyzed by the same enzyme are in conflict.
reaction_centered = reaction_centered[(reaction_centered['manipulations'] == 'down')|(reaction_centered['manipulations'] == 'Up')]
duplicate_genes = reaction_centered.groupby('gene').filter(lambda x: len(x['manipulations'].unique()) > 1)
duplicate_genes

,reaction,flux_wild,flux_over,equation,gene,manipulations
855,KARA2,"['1.117e-05', '4.467e-05']","['3.976e-04', '4.193e-04']",2ahbut_c + h_c + nadph_c --> 23dhmp_c + nadp_c,Cgl1273,Up
856,KARA1_reverse,"['3.701e-05', '7.052e-05']","['3.895e-06', '2.555e-05']",alac__S_c + h_c + nadph_c --> 23dhmb_c + nadp_c,Cgl1273,down


## 2. protein centered

In [ ]:
protein_centered = pd.read_excel('/home/sun/ETGEMS-10.20/ET_OptME/result/ET_Optme/result/ile/results_SET.xlsx')
protein_centered = protein_centered[(protein_centered['manipulations'] == 'down')|(protein_centered['manipulations'] == 'Up')]
protein_centered[protein_centered['gene'] =='Cgl1273']

,Unnamed: 0.1,Unnamed: 0,gene,enzyme_wild,enzyme_over,reaction_bio,reaction_pro,ref_e_con(g/gDW),over_e_con(g/gDW),Fold_change,enzyme_usage_over,manipulations,e1(mmol/gDW),kcat(1/h),mw(g/mg)
0,0,0,Cgl1273,"['1.742e-03', '1.864e-03']","['1.452e-02', '1.460e-02']",KARA1_reverse (0.27937066159703733): alac__S_c...,KARA2 (3.002127237423596): 2ahbut_c + h_c + na...,0.001742,0.01452,8.336,18.65,Up,0.000402,"[('KARA2', 7549.714285714285)]",36.16
